In [17]:
import requests
from bs4 import BeautifulSoup
import re
import json


In [4]:
url = "https://www.kurashiru.com/recipes/6c656bdf-d8c9-427a-b669-f6e1d9f81fcd"

In [8]:
def get_html(url, params=None, headers=None):
    try:
        resp = requests.get(url, params=params, headers=headers)
        resp.encoding = 'utf8'
        soup = BeautifulSoup(resp.text, "html.parser")
        return soup
    except Exception as e:
        return None
        

In [158]:
def get_page_info(url):
    result = {}
    soup = get_html(url)
    script = soup.find('script', string=re.compile('window.__delyKurashiruEnvironment.ssrContext'))
    if script == None:
        return
    
    try:
        script_data = re.search(r'window\.__delyKurashiruEnvironment\.ssrContext\s*=\s*({.*?});', str(script))
        if script_data == None:
            return
        data = json.loads(script_data.group(1))
        if data == None:
            return
        attributes = data["state"]["fetchVideo"]["data"]["data"]["attributes"]
        ingredients = attributes["ingredients"]
        instructions = attributes["instructions"]

        result["recipe_id"] = data["state"]["fetchVideo"]["data"]["data"]["id"]
        result["recipe_name"] = attributes["title"]
        result["video_url"] = attributes["mp4-url"]
        result["img_small_url"] = attributes["thumbnail-square-small-url"]
        result["img_normal_url"] = attributes["thumbnail-square-normal-url"]
        result["img_large_url"] = attributes["thumbnail-square-large-url"]
        result["cooking_time"] = attributes["cooking-time"]
        result["introduction"] = attributes["introduction"]
        result["expense"] = attributes["expense"]
        result["serving"] = attributes["servings-size"]
        result["rating-count"] = attributes["rating-count"]
        result["calorie"] = attributes["calorie"]

        ingredient_list = []
        for ingredient in ingredients:
            if ingredient["type"] != "ingredients":
                continue
            ingredient_dic = {}
            ingredient_dic["ingredient_id"] = ingredient["id"]
            ingredient_dic["ingredient_title"] = ingredient["name"]
            ingredient_dic["ingredient_name"] = ingredient["actual-name"]
            ingredient_dic["ingredient_amount"] = ingredient["quantity-amount"]
            ingredient_list.append(ingredient_dic)
        result["ingredient_list"] = ingredient_list

        instruction_list = []
        for instruction in instructions:
            if instruction["type"] != "instructions":
                continue
            instruction_dic = {}
            instruction_dic["index"] = instruction["sort-order"]
            instruction_dic["body"] = instruction["body"]
            instruction_list.append(instruction_dic)
        result["instruction_list"] = instruction_list
    except Exception as e:
        return
    return result
